# ScopeGreen API Interactive Tutorial

Welcome! This Google Colab notebook provides an interactive way to learn about and test the ScopeGreen API from Scope4.

**What you can do here:**
* Learn how to make API calls to search for Life Cycle Assessment (LCA) metrics.
* Experiment with different search parameters.
* See live results directly from the API.

**Prerequisites:**
* You need a ScopeGreen API Key. If you don't have one, please request access via the [Builder Form](https://scope4.dev/request-form-builder) or [Enterprise Form](https://scope4.dev/request-form-enterprise).
* Basic understanding of Python and REST APIs is helpful but not strictly required.

## 1. Setup

First, we need to install the `requests` library (if it's not already available in the Colab environment) and set up your API key securely.

In [ ]:
# Install the requests library, used for making HTTP requests
!pip install requests -q

In [ ]:
# Import necessary libraries
import requests
import json
from getpass import getpass # To securely input your API key

# --- Configuration ---
API_BASE_URL = "https://lcabench-v1-main-3f150df.d2.zuplo.dev"

# Securely get the API key from the user
api_key = getpass('Enter your ScopeGreen API Key: ')

# Set up the authorization header
headers = {
    "Authorization": f"Bearer {api_key}"
}

print("API Key set successfully (hidden for security).")

## 2. Check API Health

Let's start with a simple check to ensure the API is running.

In [ ]:
# Define the health check endpoint URL
health_url = f"{API_BASE_URL}/health"

try:
    # Make the GET request (no auth needed for this specific endpoint as per OAS, but sending anyway)
    response = requests.get(health_url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        print("API Health Check Successful!")
        print("Response:", response.json()) # Print the response body
    else:
        print(f"API Health Check Failed. Status Code: {response.status_code}")
        print("Response:", response.text) # Print raw text on failure

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

API Health Check Successful!
Response: {'status': 'healthy'}


## 3. Get Available Metrics

Find out which metrics are supported by the API.

In [ ]:
# Define the available metrics endpoint URL
available_metrics_url = f"{API_BASE_URL}/api/metrics/available"

try:
    # Make the GET request (requires auth)
    response = requests.get(available_metrics_url, headers=headers)

    # Check the status code and print results
    print(f"Request URL: {response.url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("\nAvailable Metrics:")
        # Pretty print the JSON response
        print(json.dumps(response.json(), indent=2))
    else:
        print("\nError fetching available metrics:")
        try:
            # Try to parse error JSON
            print(json.dumps(response.json(), indent=2))
        except json.JSONDecodeError:
            # Print raw text if not JSON
            print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

Request URL: https://lcabench-v1-main-3f150df.d2.zuplo.dev/api/metrics/available
Status Code: 200

Available Metrics:
{
  "metrics": [
    "Carbon footprint",
    "EF3.1 Score",
    "Land Use"
  ]
}


## 4. Search for Metrics

This is the core functionality. Let's search for LCA data for specific items.

### 4.1 Basic Search (Item Name Only)

Search for the default metric ('Carbon footprint') for 'steel'.

In [ ]:
# Define the search endpoint URL
search_url = f"{API_BASE_URL}/api/metrics/search"

# --- Parameters for the search ---
search_params = {
    "item_name": "steel" # The only required parameter
    # Other parameters will use their defaults
}

try:
    # Make the GET request with parameters
    response = requests.get(search_url, headers=headers, params=search_params)

    # Print request details and response
    print(f"Request URL: {response.url}") # Shows the full URL with query parameters
    print(f"Status Code: {response.status_code}")

    print("\nResponse Body:")
    try:
        # Pretty print JSON response
        print(json.dumps(response.json(), indent=2))
    except json.JSONDecodeError:
        # Print raw text if response is not JSON
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

Request URL: https://lcabench-v1-main-3f150df.d2.zuplo.dev/api/metrics/search?item_name=steel
Status Code: 200

Response Body:
{
  "matches": [
    {
      "rank": 1,
      "matched_name": "Steel, low-alloyed, at plant",
      "metric": {
        "name": "Carbon footprint",
        "value": 1.34,
        "unit": "kg CO2 eq / kg"
      },
      "year": 2023,
      "geography": "GLO",
      "source": "ecoinvent v3.9.1",
      "source_link": "https://ecoinvent.org/the-ecoinvent-database/",
      "conversion_info": "Unit conversion not requested."
    }
  ],
  "explanation": "The best match is 'Steel, low-alloyed, at plant' as it represents a common form of steel without specific processing details, aligning well with the general query 'steel'. It provides the requested carbon footprint metric."
}


### 4.2 Advanced Search (Multiple Parameters)

Let's try a more specific search:
* Item: `cotton t-shirt`
* Metric: `Land Use`
* Geography: `FR` (France)
* Number of Matches: `2`
* Convert Unit: `g` (grams for the functional unit)

In [ ]:
# --- Parameters for the advanced search ---
advanced_search_params = {
    "item_name": "cotton t-shirt",
    "metric": "Land Use",       # Specify the metric
    "geography": "FR",          # Specify geography
    "num_matches": 2,           # Request top 2 matches
    "unit": "g"                 # Request conversion to grams
}

try:
    # Make the GET request
    response = requests.get(search_url, headers=headers, params=advanced_search_params)

    # Print request details and response
    print(f"Request URL: {response.url}")
    print(f"Status Code: {response.status_code}")

    print("\nResponse Body:")
    try:
        print(json.dumps(response.json(), indent=2))
    except json.JSONDecodeError:
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

Request URL: https://lcabench-v1-main-3f150df.d2.zuplo.dev/api/metrics/search?item_name=cotton+t-shirt&metric=Land+Use&geography=FR&num_matches=2&unit=g
Status Code: 200

Response Body:
{
  "matches": [
    {
      "rank": 1,
      "matched_name": "Cotton T-shirt",
      "metric": {
        "name": "Land Use",
        "value": 0.0093,
        "unit": "m2a crop eq / g"
      },
      "year": 2020,
      "geography": "FR",
      "source": "Agribalyse v3.1",
      "source_link": "https://agribalyse.ademe.fr/data",
      "conversion_info": "Converted functional unit from kg to g (divide value by 1 000)."
    }
  ],
  "explanation": "The best match is 'Cotton T-shirt' from Agribalyse v3.1 (2020, FR), which directly matches the query and provides the requested 'Land Use' metric. Only one match was found despite requesting two."
}


### 4.3 Handling 'No Match' Response

Let's try searching for something unlikely to exist.

In [ ]:
# --- Parameters for a likely non-matching search ---
no_match_params = {
    "item_name": "unobtainium widget mk7"
}

try:
    # Make the GET request
    response = requests.get(search_url, headers=headers, params=no_match_params)

    # Print request details and response
    print(f"Request URL: {response.url}")
    print(f"Status Code: {response.status_code}")

    print("\nResponse Body:")
    try:
        response_data = response.json()
        print(json.dumps(response_data, indent=2))
        # Check specifically for the 'no match' message structure
        if response.status_code == 200 and "message" in response_data and "matches" not in response_data:
             print("\n---> This is the expected 'No Match Found' response.")
    except json.JSONDecodeError:
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

Request URL: https://lcabench-v1-main-3f150df.d2.zuplo.dev/api/metrics/search?item_name=unobtainium+widget+mk7
Status Code: 200

Response Body:
{
  "message": "No good match was found, retry with a different item name"
}

---> This is the expected 'No Match Found' response.


### 4.4 Handling Errors (Example: Missing Required Parameter)

The API should return an error if required parameters like `item_name` are missing. Zuplo (the API Gateway) typically handles this validation and returns a 400 Bad Request.

In [ ]:
# --- Parameters missing the required 'item_name' ---
error_params = {
    "metric": "Carbon footprint" # Missing item_name
}

try:
    # Make the GET request
    response = requests.get(search_url, headers=headers, params=error_params)

    # Print request details and response
    print(f"Request URL: {response.url}")
    print(f"Status Code: {response.status_code}") # Expecting 400 or 422

    print("\nResponse Body:")
    try:
        print(json.dumps(response.json(), indent=2))
    except json.JSONDecodeError:
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

Request URL: https://lcabench-v1-main-3f150df.d2.zuplo.dev/api/metrics/search?metric=Carbon+footprint
Status Code: 400

Response Body:
{
  "error": {
    "message": "Invalid query parameters provided",
    "details": [
      {
        "path": [
          "item_name"
        ],
        "message": "Required"
      }
    ]
  }
}


## 5. Conclusion

This notebook demonstrated the basic usage of the ScopeGreen API:
* Setting up authentication.
* Checking API health and available metrics.
* Performing basic and advanced searches for LCA data.
* Handling successful responses, 'no match' scenarios, and errors.

Feel free to modify the code cells above to experiment with different parameters and items.

**Further Resources:**
* [ScopeGreen API Documentation](https://lcabench-v1-main-3f150df.d2.zuplo.dev/docs) (Your Dev Portal Link)
* [Scope4 Website](https://scope4.dev)

For questions or support, contact tom@scope4.dev.